# Workflow
In this example, we will use McStas 3 simulation file.

## Build Pipeline (Collect Parameters and Providers)
Import the providers from ``load_mcstas_nexus`` to use the ``McStas`` simulation data workflow. <br>
``MaximumProbability`` can be manually provided to derive more realistic number of events. <br>
It is because ``weights`` are given as probability, not number of events in a McStas file. <br>

In [ ]:
from ess.nmx.mcstas import McStasWorkflow
from ess.nmx.data import small_mcstas_3_sample

from ess.nmx.types import *
from ess.nmx.reduction import NMXData, NMXReducedData, merge_panels
from ess.nmx.types import DetectorIndex


wf = McStasWorkflow()
# Replace with the path to your own file
wf[FilePath] = small_mcstas_3_sample()
wf[MaximumProbability] = 10000
wf[TimeBinSteps] = 50

To see what the workflow can produce, display it:

In [ ]:
wf

We want to reduce all three panels, so we map the relevant part of the workflow over a list of the three panels:

If you want to reduce all three panels,
map the relevant part of the workflow over a list of the three panels:

```python
# DetectorIndex selects what detector panels to include in the run
# in this case we select all three panels.
wf[NMXReducedData] = (
    wf[NMXReducedData]
    .map({DetectorIndex: sc.arange('panel', 3, unit=None)})
    .reduce(index="panel", func=merge_panels)
)
```

However, we encountered memory issue processing dataset, which is often over 10GB.
Therefore we will not merge the panels at the end of the workflow
and iter over the detector index instead.

## Build Workflow

In [ ]:
wf.visualize(NMXReducedData, graph_attr={"rankdir": "TD"}, compact=True)

## Compute Desired Types

In [ ]:
import sciline as sl
from contextlib import contextmanager
from collections.abc import Generator


@contextmanager
def temp_parameter(
    wf: sl.Pipeline, parameter_type: type, value: Any
) -> Generator[sl.Pipeline]:
    copied = wf.copy()
    copied[parameter_type] = value
    yield copied
    del copied

In [ ]:
# Data from the first detector binned by panel, pixel and timeslice
with temp_parameter(wf, DetectorIndex, 0) as temp_wf:
    binned_dg = temp_wf.compute(NMXReducedData)

binned_dg

## Export Results

``NMXReducedData`` object has a method to export the data into nexus or h5 file.

You can save the result as ``test.nxs``, for example:


In [ ]:
from ess.nmx.nexus import export_as_nxlauetof

dgs = []
for i in range(3):
    with temp_parameter(wf, DetectorIndex, i) as temp_wf:
        reduced_data = temp_wf.compute(NMXReducedData)
        dgs.append(reduced_data)
        del reduced_data

export_as_nxlauetof(*dgs, output_file="test.nxs")

In [ ]:
dgs[0]

Legacy version of the exporting method
```python
from ess.nmx.nexus import export_as_nexus

export_as_nexus(binned_dg, "test.nxs")
```

## Merge All Panels

If you simply want to compute all panels at once, you can use map/reduce on the workflow.

In [ ]:
base_wf = wf.copy()
detector_panel_ids = {DetectorIndex: sc.arange('panel', 3, unit=None)}
pipeline = base_wf.map(detector_panel_ids)
pipeline.visualize(
    sl.get_mapped_node_names(pipeline, NMXData),
    compact=True,
)

In [ ]:
dg = merge_panels(
    *pipeline.compute(sl.get_mapped_node_names(pipeline, NMXData)).values()
)
dg

## Instrument View

Pixel positions are not used for later steps,
but it is included in the coordinates for instrument view.

All pixel positions are relative to the sample position,
therefore the sample is at (0, 0, 0).

**It might be very slow or not work in the ``VS Code`` jupyter notebook editor.**

In [ ]:
import scippneutron as scn

da = dg["weights"]
da.coords["position"] = dg["position"]
# Plot one out of 100 pixels to reduce size of docs output
view = scn.instrument_view(da["id", ::100].hist(), pixel_size=0.0075)
view